In [2]:
import numpy as np
import os
import cv2
import random
import matplotlib.pyplot as plt

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping

In [3]:
# Set the path to your data
data_path = "P:\Mini_Proj_SEM_6\dataset"


In [5]:
train_data = []
val_data = []

for folder in os.listdir(data_path):
    folder_path = os.path.join(data_path, folder)
    files = os.listdir(folder_path)  # Retrieve list of files in the folder
    num_train = int(0.8 * len(files))
    files_train = random.sample(files, num_train)
    files_val = list(set(files) - set(files_train))
    
    # Process training data
    for file in files_train:
        file_path = os.path.join(folder_path, file)
        img = cv2.imread(file_path)
        if img is not None:  # Check if the image was read successfully
            img = cv2.resize(img, (224,224))
            train_data.append((img, folder))
        else:
            print(f"Failed to read image: {file_path}")

    # Process validation data
    for file in files_val:
        file_path = os.path.join(folder_path, file)
        img = cv2.imread(file_path)
        if img is not None:  # Check if the image was read successfully
            img = cv2.resize(img, (224,224))
            val_data.append((img, folder))
        else:
            print(f"Failed to read image: {file_path}")


Failed to read image: P:\Mini_Proj_SEM_6\dataset\test\Melanoma Skin Cancer Nevi and Moles
Failed to read image: P:\Mini_Proj_SEM_6\dataset\test\Systemic Disease
Failed to read image: P:\Mini_Proj_SEM_6\dataset\test\Vasculitis Photos
Failed to read image: P:\Mini_Proj_SEM_6\dataset\test\Light Diseases and Disorders of Pigmentation
Failed to read image: P:\Mini_Proj_SEM_6\dataset\test\Herpes HPV and other STDs Photos
Failed to read image: P:\Mini_Proj_SEM_6\dataset\test\Hair Loss Photos Alopecia and other Hair Diseases
Failed to read image: P:\Mini_Proj_SEM_6\dataset\test\Seborrheic Keratoses and other Benign Tumors
Failed to read image: P:\Mini_Proj_SEM_6\dataset\test\Poison Ivy Photos and other Contact Dermatitis
Failed to read image: P:\Mini_Proj_SEM_6\dataset\test\Psoriasis pictures Lichen Planus and related diseases
Failed to read image: P:\Mini_Proj_SEM_6\dataset\test\Exanthems and Drug Eruptions
Failed to read image: P:\Mini_Proj_SEM_6\dataset\test\Nail Fungus and other Nail Disea

#prepare model

In [6]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False
num_classes = len(os.listdir(data_path))
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(512, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# PREPROCESS DATA
X_train, y_train = zip(*train_data)
X_val, y_val = zip(*val_data)

X_train = preprocess_input(np.array(X_train))
X_val = preprocess_input(np.array(X_val))

le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_val_encoded = le.transform(y_val)

y_train_one_hot = to_categorical(y_train_encoded, num_classes)
y_val_one_hot = to_categorical(y_val_encoded, num_classes)




94765736/94765736 [==============================] - 25s 0us/step



ValueError: not enough values to unpack (expected 2, got 0)

In [ ]:
custom_early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=10, 
    min_delta=0.001, 
    mode='min'
)

EPOCHS = 20
BATCH_SIZE = 64
history = model.fit(X_train, y_train_one_hot, validation_data=(X_val, y_val_one_hot),
                   epochs=EPOCHS, batch_size=BATCH_SIZE, callbacks=[custom_early_stopping])


In [ ]:
model.save('D:\SEM 6\mini-project\models\my_model1.h5')


In [ ]:
# Plot training and validation losses
plt.plot(history.history['loss'], label='Training loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.title('Training and Validation Losses')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Plot training and validation accuracy
plt.plot(history.history['accuracy'], label='Training accuracy')
plt.plot(history.history['val_accuracy'], label='Validation accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# PREDICT
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model('D:\SEM 6\mini-project\models\my_model1.h5')

In [ ]:
real_label = []
predicted_class = []

test_path = 'P:\\Mini_Proj_SEM_6\\dataset\\test'

for folder in os.listdir(test_path):
    folder_path = os.path.join(test_path, folder)
    for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file)
        img = cv2.imread(file_path)
        img = cv2.resize(img, (224,224))
        img = preprocess_input(np.array([img]))  # Add an extra dimension for batching

        predictions = model.predict(img)
        real_label.append(folder)
        predicted_class_index = np.argmax(predictions)
        predicted_class.append(le.classes_[predicted_class_index])

In [ ]:
# Plot confusion matrix
from sklearn.metrics import confusion_matrix
import seaborn as sns

conf_matrix = confusion_matrix(real_label, predicted_class)
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()